
# End-to-End Experimentation & Causal Inference Tutorial (with CATE)
**From first principles → advanced methods.**  
You’ll build cohorts, size an experiment (power), run A/B with CUPED, estimate causal effects with back-door OLS and IV (2SLS), handle exposure bias with IPW, and **estimate heterogeneous treatment effects (CATE)** using meta-learners with Random Forests (a practical stand‑in for causal forests).

**What you’ll learn**  
- Potential outcomes, counterfactuals, confounders, instruments, exposure bias  
- Cohorting and experiment design basics  
- Power analysis & variance reduction (CUPED)  
- Causal estimation: back-door OLS and 2SLS IV  
- **CATE estimation** with T‑learner / S‑learner / X‑learner (Random Forests)  
- Uplift targeting and policy evaluation ideas

> Note: True “Causal Forests” are implemented in packages like `econml` or `grf`. Here we implement **meta‑learner** approaches with `sklearn` Random Forests which deliver forest‑style **nonlinear CATE** approximations that work well in practice and are easy to run anywhere.



## 1. Setup & Synthetic Data
We simulate a realistic product setting with:
- **Confounding** (treatment is more likely among highly active users)
- A **randomized encouragement** `Z` (valid instrument for 2SLS)
- **Exposure** counts that correlate with outcomes (exposure bias)
- A **true structural effect** `tau = 1.2` so we can judge estimators

Run the next cell to generate the dataset.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant, Logit
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.weightstats import DescrStatsW

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rng = np.random.default_rng(42)

# ---- simulate users ----
N = 50_000
signup_month = rng.integers(1, 7, size=N)
segment = rng.choice(["casual", "regular", "vip"], size=N, p=[0.5, 0.4, 0.1])
age = rng.normal(35, 10, size=N).clip(18, 75)
activity_score = rng.gamma(shape=2.0, scale=2.0, size=N)
u_latent = rng.normal(0, 1, size=N)

# pre-experiment metric (for CUPED)
pre_metric = 5 + 0.25 * activity_score + 0.02 * (age - 35) + 0.8 * u_latent + rng.normal(0, 1.0, N)

# Instrument: randomized encouragement
Z = rng.binomial(1, 0.5, size=N)

# Non-random treatment: encouraged + latent inclination drive T
logit_T = -0.2 + 1.1 * Z + 0.6 * u_latent + 0.15 * (activity_score > 2.5)
p_T = 1 / (1 + np.exp(-logit_T))
T = rng.binomial(1, p_T, size=N)

# Exposure (dose)
lambda_exposure = 0.8 + 1.3 * T + 0.25 * activity_score
exposure = rng.poisson(lam=np.maximum(lambda_exposure, 0.1))

# Structural outcome
tau = 1.2           # true treatment effect
beta_exp = 0.15     # effect of exposure (mediator-like)
theta_u = 1.0       # unobserved component
theta_cov = 0.02
theta_seg = {"casual": 0.0, "regular": 0.4, "vip": 1.0}
seg_eff = np.array([theta_seg[s] for s in segment])

Y = (
    2.0 + tau*T + beta_exp*exposure
    + theta_cov*(age - 35)
    + 0.3*activity_score
    + seg_eff + theta_u*u_latent
    + rng.normal(0, 1.0, N)
)

df = pd.DataFrame({
    "signup_month": signup_month,
    "segment": segment,
    "age": age,
    "activity_score": activity_score,
    "pre_metric": pre_metric,
    "Z": Z,
    "T": T,
    "exposure": exposure,
    "Y": Y
})
df.head()



## 2. Cohorts for baseline sanity & stratification
Cohorts let you check balance, set up stratified designs, and monitor drift across comparable slices.


In [ ]:

cohort_cols = ["signup_month", "segment"]
cohort_summary = (
    df.groupby(cohort_cols)
      .agg(n=("Y","size"),
           mean_Y=("Y","mean"),
           mean_pre=("pre_metric","mean"),
           treat_rate=("T","mean"))
      .reset_index()
      .sort_values(cohort_cols)
)
cohort_summary.head(12)



## 3. Power analysis (two-sample t-test approximation)
Pick a minimum detectable effect (MDE) and estimate **n per arm**. Then draw a **power curve** for intuition.


In [ ]:

effect_target = 0.6
std_est = df["Y"].std()
cohen_d = effect_target / std_est

n_per_group = int(np.ceil(TTestIndPower().solve_power(effect_size=cohen_d, alpha=0.05, power=0.8)))
print("Needed per group (approx):", n_per_group)

# power vs true effect, at this fixed n
effects = np.linspace(0.2, 1.2, 6)
powers = [TTestIndPower().power(effect_size=e/std_est, nobs1=n_per_group, alpha=0.05) for e in effects]

plt.figure(figsize=(6,4))
plt.plot(effects, powers, marker="o")
plt.title(f"Power vs Effect (n/group ≈ {n_per_group})")
plt.xlabel("True Effect on Y")
plt.ylabel("Power")
plt.grid(True)
plt.tight_layout()
plt.show()



## 4. A/B test and CUPED variance reduction
We simulate a randomized **`ab_group`** on a 20% subsample, measure the naive diff‑in‑means, then apply **CUPED** using the correlated **pre_metric** to reduce variance.


In [ ]:

rng = np.random.default_rng(7)
mask_ab = rng.random(df.shape[0]) < 0.2
df_ab = df.loc[mask_ab].copy()

df_ab["ab_group"] = rng.binomial(1, 0.5, size=df_ab.shape[0])
delta_true = 0.8
df_ab["Y_ab"] = df_ab["Y"] + delta_true*df_ab["ab_group"] + rng.normal(0, 0.5, df_ab.shape[0])

grp0 = df_ab.loc[df_ab.ab_group==0, "Y_ab"]
grp1 = df_ab.loc[df_ab.ab_group==1, "Y_ab"]
diff = grp1.mean() - grp0.mean()
print("Naive diff:", diff)

# CUPED
pre = df_ab["pre_metric"]
theta = np.cov(df_ab["Y_ab"], pre, ddof=1)[0,1] / np.var(pre, ddof=1)
Y_cuped = df_ab["Y_ab"] - theta*(pre - pre.mean())

grp0_c = Y_cuped[df_ab.ab_group==0]
grp1_c = Y_cuped[df_ab.ab_group==1]
diff_c = grp1_c.mean() - grp0_c.mean()
print("CUPED diff:", diff_c)

# plots
plt.figure(figsize=(6,4))
plt.hist(grp0, bins=40, alpha=0.6, label="Control Y_ab", density=True)
plt.hist(grp1, bins=40, alpha=0.6, label="Variant Y_ab", density=True)
plt.title("A/B outcome distributions (naive)")
plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.hist(grp0_c, bins=40, alpha=0.6, label="Control (CUPED)", density=True)
plt.hist(grp1_c, bins=40, alpha=0.6, label="Variant (CUPED)", density=True)
plt.title("A/B outcome distributions (CUPED-adjusted)")
plt.legend(); plt.tight_layout(); plt.show()



## 5. Causal estimation: back‑door OLS and 2SLS IV
- **Back‑door OLS:** control for observed confounders \(X\) that block spurious paths.  
- **IV / 2SLS:** use a randomized encouragement `Z` that shifts `T` but has no direct path to `Y` except via `T`.


In [ ]:

# Naive OLS
ols_naive = OLS(df["Y"], add_constant(df[["T"]])).fit()

# Back-door OLS
enc = OneHotEncoder(drop="first", sparse=False, handle_unknown="ignore")
seg_oh = enc.fit_transform(df[["segment"]])
seg_cols = [f"segment_{c}" for c in enc.categories_[0][1:]]
Xb = pd.DataFrame(seg_oh, columns=seg_cols)
Xb["T"] = df["T"].values
Xb["age"] = df["age"].values
Xb["activity_score"] = df["activity_score"].values
ols_backdoor = OLS(df["Y"], add_constant(Xb)).fit()

# 2SLS (manual)
X1 = pd.DataFrame(seg_oh, columns=seg_cols)
X1["Z"] = df["Z"].values
X1["age"] = df["age"].values
X1["activity_score"] = df["activity_score"].values
stage1 = OLS(df["T"], add_constant(X1)).fit()
T_hat = stage1.fittedvalues

X2 = pd.DataFrame(seg_oh, columns=seg_cols)
X2["T_hat"] = T_hat.values
X2["age"] = df["age"].values
X2["activity_score"] = df["activity_score"].values
stage2 = OLS(df["Y"], add_constant(X2)).fit()

print("Naive OLS (Y~T):", round(ols_naive.params["T"],3))
print("Back-door OLS:", round(ols_backdoor.params["T"],3))
print("2SLS IV:", round(stage2.params["T_hat"],3))
print("True tau:", 1.2)



## 6. Exposure bias & IPW correction
If high‑activity users both **get treated more** and **perform better**, naive estimates inflate the effect. Estimate the **propensity** \(e(X)=P(T=1|X)\) and compute an **IPW ATE**.


In [ ]:

Xp = pd.DataFrame(seg_oh, columns=seg_cols)
Xp["age"] = df["age"].values
Xp["activity_score"] = df["activity_score"].values

logit = Logit(df["T"], add_constant(Xp)).fit(disp=False)
ps = logit.predict(add_constant(Xp)).clip(1e-3, 1-1e-3)

w = np.where(df["T"]==1, 1/ps, 1/(1-ps))
ds1 = DescrStatsW(df.loc[df.T==1,"Y"], w[df.T==1], ddof=1)
ds0 = DescrStatsW(df.loc[df.T==0,"Y"], w[df.T==0], ddof=1)

ate_naive = df.loc[df.T==1,"Y"].mean() - df.loc[df.T==0,"Y"].mean()
ate_ipw = ds1.mean - ds0.mean
print("Naive diff (T groups):", round(ate_naive,3))
print("IPW ATE:", round(ate_ipw,3))
print("True tau:", 1.2)



## 7. Heterogeneous Treatment Effects (CATE)
We now estimate **CATE(x) = E[Y(1)−Y(0) | X=x]**.  
We’ll implement three **meta‑learner** strategies using Random Forests (nonparametric, interaction‑rich):

- **T‑learner (two models):** fit \(\mu_1(x)=E[Y|X=x,T=1]\) and \(\mu_0(x)=E[Y|X=x,T=0]\), then \(\widehat{\tau}(x)=\mu_1(x)-\mu_0(x)\).
- **S‑learner (single model):** fit \(m(x,t)=E[Y|X=x,T=t]\) on features `[X, T]`, then predict \(m(x,1)-m(x,0)\).
- **X‑learner:** (a refinement) first build T‑learner, compute pseudo-outcomes \(D_1=Y-\mu_0(X)\) for treated and \(D_0=\mu_1(X)-Y\) for control; then learn separate models mapping \(X\to D\) and combine.

These recover **nonlinear**, segment‑specific effects and approximate causal forests behavior without extra dependencies.


In [ ]:

# Features X
X_base = pd.DataFrame(seg_oh, columns=seg_cols)
X_base["age"] = df["age"].values
X_base["activity_score"] = df["activity_score"].values
X_base["pre_metric"] = df["pre_metric"].values
X = X_base.values
y = df["Y"].values
t = df["T"].values

X_tr, X_te, y_tr, y_te, t_tr, t_te = train_test_split(X, y, t, test_size=0.3, random_state=123, stratify=t)

# --- T-learner ---
rf_t1 = RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1)
rf_t0 = RandomForestRegressor(n_estimators=200, random_state=1, n_jobs=-1)

rf_t1.fit(X_tr[t_tr==1], y_tr[t_tr==1])
rf_t0.fit(X_tr[t_tr==0], y_tr[t_tr==0])

mu1 = rf_t1.predict(X_te)
mu0 = rf_t0.predict(X_te)
cate_T = mu1 - mu0

# --- S-learner ---
rf_s = RandomForestRegressor(n_estimators=300, random_state=2, n_jobs=-1)
X_s_tr = np.c_[X_tr, t_tr]
X_s_te_1 = np.c_[X_te, np.ones_like(t_te)]
X_s_te_0 = np.c_[X_te, np.zeros_like(t_te)]
rf_s.fit(X_s_tr, y_tr)
cate_S = rf_s.predict(X_s_te_1) - rf_s.predict(X_s_te_0)

# --- X-learner ---
# Step 1: use T-learner mu0/mu1 from above to form pseudo-outcomes
D1 = y_tr[t_tr==1] - rf_t0.predict(X_tr[t_tr==1])   # for treated
D0 = rf_t1.predict(X_tr[t_tr==0]) - y_tr[t_tr==0]   # for control

rf_x1 = RandomForestRegressor(n_estimators=200, random_state=3, n_jobs=-1)
rf_x0 = RandomForestRegressor(n_estimators=200, random_state=4, n_jobs=-1)
rf_x1.fit(X_tr[t_tr==1], D1)
rf_x0.fit(X_tr[t_tr==0], D0)

# Propensity for weighting (simple logistic on train split)
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)
lr.fit(X_tr, t_tr)
p_te = np.clip(lr.predict_proba(X_te)[:,1], 1e-3, 1-1e-3)

tau1 = rf_x1.predict(X_te)   # estimate if treated
tau0 = rf_x0.predict(X_te)   # estimate if control
cate_X = p_te * tau0 + (1 - p_te) * tau1

# Summaries
def summarize(arr, name):
    q = np.quantile(arr, [0.05, 0.25, 0.5, 0.75, 0.95])
    print(f"{name} CATE ~ mean={arr.mean():.3f}, median={q[2]:.3f}, IQR=({q[1]:.3f},{q[3]:.3f}), 5-95%: ({q[0]:.3f},{q[4]:.3f})")

summarize(cate_T, "T-learner")
summarize(cate_S, "S-learner")
summarize(cate_X, "X-learner")

plt.figure(figsize=(6,4))
plt.hist(cate_T, bins=50, alpha=0.6, label="T-learner", density=True)
plt.hist(cate_S, bins=50, alpha=0.6, label="S-learner", density=True)
plt.hist(cate_X, bins=50, alpha=0.6, label="X-learner", density=True)
plt.title("Estimated CATE distributions (test set)")
plt.xlabel("Estimated uplift")
plt.ylabel("Density")
plt.legend(); plt.tight_layout(); plt.show()



## 8. Targeting by uplift (simple policy)
Use CATE scores to target a top fraction of users. Compare average outcomes within **absolutely randomized** test cells if available; in synthetic offline evaluation, we proxy gains by **predicted** uplift and sanity‑check monotonicity across score deciles.


In [ ]:

# Rank by X-learner CATE (often most stable)
order = np.argsort(-cate_X)
deciles = np.array_split(order, 10)

avg_cate_by_decile = [cate_X[idx].mean() for idx in deciles]
plt.figure(figsize=(6,4))
plt.plot(range(1,11), avg_cate_by_decile, marker="o")
plt.title("Average predicted uplift by decile (X-learner)")
plt.xlabel("Decile (1=highest uplift)")
plt.ylabel("Mean predicted uplift")
plt.grid(True); plt.tight_layout(); plt.show()

print("Top-10% mean predicted uplift:", avg_cate_by_decile[0])



## 9. Key takeaways & next steps
- **Design > analysis:** randomize where possible; stratify by cohorts; track SRM; use CUPED for precision.  
- **Back‑door vs IV:** control observed confounders; if unobservables remain but an instrument exists, 2SLS helps.  
- **Exposure bias:** fix via design (eligibility caps/strata) or analysis (IPW, front‑door, dose models).  
- **CATE:** meta‑learners with Random Forests give powerful **nonlinear heterogeneity** with minimal dependencies.  
- **Next:** add honest splitting, cross‑fitting, doubly‑robust learners (e.g., `EconML` CausalForestDML / DR Learners), multiple‑testing control for many segments, and online uplift experiments (interleaving, Thompson sampling).

---

### Repro notes
- This notebook avoids non‑standard packages so it’s portable.  
- True causal forests are available in `econml` (`CausalForestDML`) or `grf` (R). Swap our meta‑learners with those for state‑of‑the‑art CATE.
